<h1 style="color:crimson">Proyecto final</h1>

#### Importacoón de librerías requedidas

In [1]:
#Se importan todas las librerías necesarias
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import scipy.optimize as optimize
import statsmodels.api as sm

#### Extración de mis 7 acciones del IPC

In [2]:
#Se lee el archivo del IPC Histórico y se parsea en un Pandas Dataframe
ipc_path = "ipc.xlsx"
market = pd.read_excel(ipc_path, index_col=0)
market.columns = [c.replace(" MF Equity", "").replace(" INDEX", "").replace("*", "") for c in market.columns]
Portafolio = market[['KIMBERA','CEMEXCPO', 'FEMSAUBD', 'WALMEX', 'GFNORTEO', 'LABB', 'OMAB','MEXBOL']].sort_index(ascending=True)
#Portafolio2 = market[['BIMBOA','CEMEXCPO','MEXBOL']].sort_index(ascending=True)
Portafolio = Portafolio.pct_change(1)
Portafolio.dropna(axis=0,inplace=True)
Portafolio


,KIMBERA,CEMEXCPO,FEMSAUBD,WALMEX,GFNORTEO,LABB,OMAB,MEXBOL
DATE,,,,,,,,
2016-01-04,-0.029973,-0.033257,-0.024810,-0.014946,-0.033688,0.040433,-0.022007,-0.020099
2016-01-05,-0.006895,-0.014235,0.001142,-0.004202,-0.000763,-0.044414,-0.002324,-0.001710
2016-01-06,0.006171,-0.002407,-0.009062,-0.006564,-0.000981,-0.008715,0.005516,-0.008337
2016-01-07,0.000767,-0.078408,-0.007866,-0.056395,-0.022809,-0.029304,-0.013288,-0.024696
2016-01-08,-0.013023,-0.060209,-0.016308,0.026007,-0.007706,-0.015094,-0.016185,-0.009744
...,...,...,...,...,...,...,...,...
2019-12-30,-0.006964,-0.009682,0.002012,-0.026240,-0.035990,-0.032292,-0.016583,-0.013647
2019-12-31,-0.025974,-0.011173,-0.003347,-0.000554,0.006190,0.006997,0.004889,-0.002668
2020-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Cálculo de rendimientos esperados, Varianza y desviación

In [3]:
Rendimuent_dia = Portafolio.mean()
Rendimuent_anio = Portafolio.mean() * 252
Varianza_dia = Portafolio.var()
Varianza_anio = Portafolio.var() * 255
Desviación_día = Portafolio.std()
Desviación_anio =Portafolio.std() * 252
Matriz_rendimientos = pd.DataFrame([Rendimuent_dia,Rendimuent_anio,Varianza_dia,Varianza_anio,Desviación_día,Desviación_anio ],index=('Rendimiento diario', 'Rendimiento anual', 'Varianza promedio diaria', 'Varianza promedio anual', 'Desviación estandar diaria', "Desviación estandar anual"))
Matriz_rendimientos.drop('MEXBOL',axis=1,inplace=True)
Matriz_rendimientos



,KIMBERA,CEMEXCPO,FEMSAUBD,WALMEX,GFNORTEO,LABB,OMAB
Rendimiento diario,0.000066,0.000020,0.000185,0.000327,0.000313,0.000521,0.000631
Rendimiento anual,0.016722,0.005054,0.046561,0.082426,0.078811,0.131383,0.158900
Varianza promedio diaria,0.000245,0.000411,0.000128,0.000215,0.000341,0.000463,0.000253
Varianza promedio anual,0.062491,0.104704,0.032732,0.054868,0.087047,0.118057,0.064419
Desviación estandar diaria,0.015655,0.020263,0.011330,0.014669,0.018476,0.021517,0.015894
Desviación estandar anual,3.944935,5.106379,2.855054,3.696483,4.655944,5.422206,4.005330


#### Modelo CAPM

In [4]:
y = Portafolio['MEXBOL']
X = Portafolio.drop('MEXBOL',axis=1,inplace=False)
X1 = sm.add_constant(X)
model = sm.OLS(y, X1)
results = model.fit()
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                 MEXBOL   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.849
Method:                 Least Squares   F-statistic:                     840.0
Date:                Mon, 22 Jun 2020   Prob (F-statistic):               0.00
Time:                        01:13:54   Log-Likelihood:                 4502.5
No. Observations:                1045   AIC:                            -8989.
Df Residuals:                    1037   BIC:                            -8949.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0001      0.000     -1.230      0.2

In [5]:
Betas = pd.read_html(results.summary().tables[1].as_html(), header=0, index_col=0)[0]
Betas.drop(['std err','t','P>|t|','[0.025','0.975]'],axis=1,inplace=True)
Betas.columns= ['Beta']
Betas = Betas.tail(len(Matriz_rendimientos.transpose()))
Betas

,Beta
KIMBERA,0.0617
CEMEXCPO,0.1325
FEMSAUBD,0.2215
WALMEX,0.1236
GFNORTEO,0.1539
LABB,0.0218
OMAB,0.0767


####  Matriz de covarianza

In [6]:
covarianza = Portafolio.cov()
covarianza.drop('MEXBOL',axis=1,inplace=True)
covarianza = covarianza.head(7)
covarianza

,KIMBERA,CEMEXCPO,FEMSAUBD,WALMEX,GFNORTEO,LABB,OMAB
KIMBERA,0.000245,0.000084,0.000038,0.000046,0.000076,0.000070,0.000057
CEMEXCPO,0.000084,0.000411,0.000045,0.000048,0.000130,0.000085,0.000074
FEMSAUBD,0.000038,0.000045,0.000128,0.000042,0.000063,0.000029,0.000049
WALMEX,0.000046,0.000048,0.000042,0.000215,0.000082,0.000047,0.000051
GFNORTEO,0.000076,0.000130,0.000063,0.000082,0.000341,0.000079,0.000096
LABB,0.000070,0.000085,0.000029,0.000047,0.000079,0.000463,0.000038
OMAB,0.000057,0.000074,0.000049,0.000051,0.000096,0.000038,0.000253


#### Coeficiente de Sharpe

In [7]:
pesos = []
acciones = len(Portafolio.columns)-1
for i in range(acciones):
   pesos.append(1/acciones)
pesos = np.array(pesos)
pesos

array([0.14285714, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
       0.14285714, 0.14285714])

In [8]:
def sharpe(w):
    '''Sharpe ratio con rf de 5%'''
    return (mu(w,r) - rf) / sigma(w,C)


def mu(w,r):
    '''Rendimiento portafolio '''
    return sum(w * r * 252)


def sigma(w, C):
    '''Desv STD portafolio '''
    return np.dot(w,np.dot(C,w)) ** (1/2) * 252 ** (1/2)


def neg_sharpe(w):
    '''Sharpe ratio negativo'''
    return -sharpe(w)

r = Portafolio.drop('MEXBOL',axis=1,inplace=False).mean()
C = Portafolio.drop('MEXBOL',axis=1,inplace=False).cov()
rf = .05
sharp_pesos_iguales=sharpe(pesos)
sharp_pesos_iguales

0.1558105323734628

#### Coeficiente de Sharpe óptimo

In [9]:
def apply_sum_constraint(inputs):
   total = 1- np.sum(inputs)
   return total

my_constraints = ({'type': 'eq', "fun": apply_sum_constraint })


result = optimize.minimize(neg_sharpe,
                     pesos,
                     method='SLSQP',
                     bounds=((0, 1.0), (0, 1.0), (0, 1.0),(0, 1.0),(0, 1.0),(0, 1.0),(0, 1.0)),
                     options={'disp': True},
                     constraints=my_constraints)

result2 = optimize.minimize(neg_sharpe,
                     pesos,
                     method='SLSQP',
                     bounds=((-1.0, 1.0), (-1.0, 1.0), (-1.0, 1.0),(-1.0, 1.0),(-1.0, 1.0),(-1.0, 1.0),(-1.0, 1.0)),
                     options={'disp': True},
                     constraints=my_constraints)
print(result)
print(result2)
optimal_w = result["x"]


Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.4727155469293623
            Iterations: 5
            Function evaluations: 45
            Gradient evaluations: 5
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.5938342351173456
            Iterations: 11
            Function evaluations: 99
            Gradient evaluations: 11
     fun: -0.4727155469293623
     jac: array([ 0.08271054,  0.18090692, -0.10334753, -0.23848723, -0.12759894,
       -0.23904209, -0.23911381])
 message: 'Optimization terminated successfully.'
    nfev: 45
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([1.48180029e-16, 0.00000000e+00, 0.00000000e+00, 4.26736838e-02,
       5.35392536e-17, 2.46073936e-01, 7.11252380e-01])
     fun: -0.5938342351173456
     jac: array([-0.09250651, -0.09264326, -0.09199545, -0.09220097, -0.09209987,
       -0.09261157, -0.15191377])
 message: 'Optimization terminated 

In [10]:
sharp_pesos_optimos=sharpe(optimal_w)
sharp_pesos_optimos



0.4727155469293623

#### VaR

In [11]:
Monto = 10_000_000
Nivel_confianza = 0.95
Var_optima = np.dot(optimal_w.T, np.dot(np.array(covarianza), optimal_w))
vol=[]
for i in range(len(covarianza)):
    vol.append(math.sqrt(covarianza[covarianza.index.values[i]][covarianza.index.values[i]]))
Betas['Volatilidad diaria' ] = vol
Betas['Vector de posición'] = Betas['Beta']*Monto
Betas['VaR Individual diaria'] = Betas['Vector de posición']*Betas['Volatilidad diaria']
Betas

,Beta,Volatilidad diaria,Vector de posición,VaR Individual diaria
KIMBERA,0.0617,0.015655,617000.0,9658.828763
CEMEXCPO,0.1325,0.020263,1325000.0,26849.014038
FEMSAUBD,0.2215,0.011330,2215000.0,25095.015590
WALMEX,0.1236,0.014669,1236000.0,18130.370195
GFNORTEO,0.1539,0.018476,1539000.0,28434.512848
LABB,0.0218,0.021517,218000.0,4690.638452
OMAB,0.0767,0.015894,767000.0,12190.824913


In [12]:
VaR_Total = Betas['VaR Individual diaria'].sum()
VaR_Total

125049.20479877561